In [1]:
# copyright ############################### #
# This file is part of the Xtrack Package.  #
# Copyright (c) CERN, 2021.                 #
# ######################################### #


import ducktrack as dtk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from tqdm import tqdm

import xtrack as xt
import xobjects as xo
import xpart as xp


beta_rel = 0.305
#beta_rel = 0.106

gamma = 1.050
#gamma = 1.006


current=2.4
length = 1.5 # m cooler length
r_beam=25*1e-3

mass0=938.27208816*1e6 #ev/c^2

T_perp = 100e-3 # <E> [eV] = kb*T
T_l =  1e-3 # <E> [eV]
magnetic_field = 0.060 # T for AD
B_ratio=1e-10



c=299792458.0

p0c = mass0*beta_rel*gamma #eV/c



circumference = 182.43280000000 #m
T = circumference/(c*beta_rel)
s_per_turn = T


beta_x=10 
beta_y=4

#disp_x=0.12

                                        


In [2]:


offset_dp=0.01
num_particles=1
delta_values=[5e-4,-5e-4]
disp_values=np.linspace(-17.5, 17.5, num=40)
# Initialize empty list for sum of cooling times
sum_cooling_time = []
for delta_loop in (delta_values):
    cooling_time=[]
    for disp in tqdm(disp_values):
        
        disp_x=disp
        

        arc=xt.LinearTransferMatrix(Q_x=5.44, Q_y=5.42,
                                    beta_x_0=beta_x, beta_x_1=beta_x,
                                    beta_y_0=beta_y,  beta_y_1=beta_y,
                                    alpha_x_0=0,   alpha_x_1=0,
                                    alpha_y_0=0,   alpha_y_1=0,
                                    disp_x_0=disp_x,disp_x_1=disp_x,
                                    disp_y_0=0,    disp_y_1=0,
                                    beta_s=1*1e40,
                                    Q_s=0,
                                    chroma_x=0.0, chroma_y=0)

        delta_single=delta_loop
        
        particles = xp.Particles(
                
                mass0=mass0,
                p0c=p0c,
                x=disp_x*delta_single,
                px=0,
                y=0,
                py=0,
                delta=delta_single,
                zeta=0)
        
    
        cooler = xt.ElectronCooler(current=current,length=length,r_beam=r_beam,
                                                T_perp=T_perp,T_l=T_l,
                                                magnetic_field=magnetic_field,B_ratio=B_ratio,
                                                Neutralisation_space_charge=1,
                                                offset_dp=offset_dp)

            

        num_turns=int(2*1e5)


        # initialize arrays
        x, px, delta, action_x = [], [], [], []
        
        cutoff=0.75
        # loop over turns
        for i in tqdm(range(num_turns), desc='Cooling Progress', leave=False):
            if abs(particles.delta) < abs(cutoff* delta_single):
                        cooling_time.append(i*s_per_turn)
                        break

            
            # track particle
            arc.track(particles)
            cooler.track(particles)
        
    
        if abs(particles.delta) >= abs(cutoff* delta_single):
            cooling_time.append(num_turns * s_per_turn)  # Append default value if cooling condition is not met

    # Create a folder if it doesn't exist
    import os
    if not os.path.exists('data_results'):
        os.makedirs('data_results')

    # Save the variables to a compressed .npz file in the folder
    np.savez(f'data_results/cooling_time_dispersion{delta_loop}.npz', disp_values=disp_values, cooling_time=cooling_time)
    # Add cooling_time to sum_cooling_time
    sum_cooling_time.append(cooling_time)

    total_cooling_time=np.sum(sum_cooling_time, axis=0)


  0%|          | 0/40 [00:00<?, ?it/s]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure()
plt.plot(disp_values,cooling_time)
plt.xlabel(r'$D_x$ [m]')
plt.ylabel('cooling time [s]')
plt.title(r'cooling time vs dispersion')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure()


plt.plot(disp_values, total_cooling_time, label='Sum',color='red')
for delta in delta_values:
# Your code to generate disp_values and cooling_time
    data = np.load(f'data_results/cooling_time_dispersion{delta}.npz')
    disp_values = data['disp_values']
    cooling_time = data['cooling_time']
    plt.plot(disp_values,cooling_time,label=f'dp/p={delta}')
    
    
    plt.xlabel(r'$D_x$ [m]')
    plt.ylabel('cooling time [s]')
    plt.title(f'cooling time vs dispersion(offset_dp={offset_dp})')
    
    plt.legend()

plt.show()
